***Total: 42 points***

Complete this homework by writing R code to complete the following tasks. Keep in mind:

i. Empty chunks have been included where code is required
ii. This homework requires use of data files:

  - `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg` (Problems 1, 2)
  - `GIAB_highconf_v.3.3.2.vcf.gz` (Problem 3)
  
iv. You will be graded on your code and output results. The assignment is worth 42 points total; partial credit can be awarded.

For additional resources, please refer to these links:  
Problems 1 & 2:  
  - https://www.bioconductor.org/packages/devel/bioc/vignettes/plyranges/inst/doc/an-introduction.html
  - https://bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html  
Problem 3:  
  - https://bioconductor.org/packages/release/bioc/vignettes/Rsamtools/inst/doc/Rsamtools-Overview.pdf  
Problem 4: 
  - https://bioconductor.org/packages/release/bioc/vignettes/VariantAnnotation/inst/doc/VariantAnnotation.pdf  

# Problem 1: Overlaps between genomic regions and copy number alterations. (14 points total)

### Preparation
Load copy number segment results as shown in *2.1 BED format* of *Lecture16_GenomicData.Rmd*. You will use the same file as in the lecture notes, `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg`. Here is code to get you started.

In [1]:
#load packages
suppressPackageStartupMessages({
    library(tidyverse)
    library(GenomicRanges)
    library(plyranges)
    library(VariantAnnotation)
})

In [24]:
segs <- read.delim("/Users/sgrizzi/Desktop/TFCB_data/BRCA.genome_wide_snp_6_broad_Level_3_scna.seg", as.is = TRUE)
mode(segs$Chromosome) <- "character" 
segs[segs$Chromosome == 23, "Chromosome"] <- "X"
segs.gr <- as(segs, "GRanges")

head(segs)

,Sample,Chromosome,Start,End,Num_Probes,Segment_Mean
,<chr>,<chr>,<int>,<int>,<int>,<dbl>
1,TCGA-3C-AAAU-10A-01D-A41E-01,1,3218610,95674710,53225,0.0055
2,TCGA-3C-AAAU-10A-01D-A41E-01,1,95676511,95676518,2,-1.6636
3,TCGA-3C-AAAU-10A-01D-A41E-01,1,95680124,167057183,24886,0.0053
4,TCGA-3C-AAAU-10A-01D-A41E-01,1,167057495,167059336,3,-1.0999
5,TCGA-3C-AAAU-10A-01D-A41E-01,1,167059760,181602002,9213,-0.0008
6,TCGA-3C-AAAU-10A-01D-A41E-01,1,181603120,181609567,6,-1.2009


### a. Find the segments in `segs.gr` that have *any* overlap with the region `chr8:128,746,347-128,755,810` (4 points)
Print out the first five unique TCGA IDs.

In [3]:
segs.gr <- segs %>%
  GRanges() %>%
  print()

GRanges object with 284458 ranges and 3 metadata columns:
           seqnames              ranges strand |                 Sample
              <Rle>           <IRanges>  <Rle> |            <character>
       [1]        1    3218610-95674710      * | TCGA-3C-AAAU-10A-01D..
       [2]        1   95676511-95676518      * | TCGA-3C-AAAU-10A-01D..
       [3]        1  95680124-167057183      * | TCGA-3C-AAAU-10A-01D..
       [4]        1 167057495-167059336      * | TCGA-3C-AAAU-10A-01D..
       [5]        1 167059760-181602002      * | TCGA-3C-AAAU-10A-01D..
       ...      ...                 ...    ... .                    ...
  [284454]       19     284018-58878226      * | TCGA-Z7-A8R6-01A-11D..
  [284455]       20     455764-62219837      * | TCGA-Z7-A8R6-01A-11D..
  [284456]       21   15347621-47678774      * | TCGA-Z7-A8R6-01A-11D..
  [284457]       22   17423930-49331012      * | TCGA-Z7-A8R6-01A-11D..
  [284458]        X   3157107-154905589      * | TCGA-Z7-A8R6-01A-11D..
      

In [4]:
myGRange8 <- GRanges(seqnames = "8",
                    ranges = IRanges(start = 128746346, end = 128755810))
print(myGRange8)

GRanges object with 1 range and 0 metadata columns:
      seqnames              ranges strand
         <Rle>           <IRanges>  <Rle>
  [1]        8 128746346-128755810      *
  -------
  seqinfo: 1 sequence from an unspecified genome; no seqlengths


In [5]:
segs.overlap <- find_overlaps(segs.gr, myGRange8)
segs.overlap

GRanges object with 2200 ranges and 3 metadata columns:
         seqnames              ranges strand |                 Sample
            <Rle>           <IRanges>  <Rle> |            <character>
     [1]        8  40440599-145232496      * | TCGA-3C-AAAU-10A-01D..
     [2]        8  87789846-145232496      * | TCGA-3C-AAAU-01A-11D..
     [3]        8    617667-145232496      * | TCGA-3C-AALI-10A-01D..
     [4]        8  62096568-145232496      * | TCGA-3C-AALI-01A-11D..
     [5]        8  82383848-145232496      * | TCGA-3C-AALJ-10A-01D..
     ...      ...                 ...    ... .                    ...
  [2196]        8  55674507-145232496      * | TCGA-XX-A89A-01A-11D..
  [2197]        8    617667-145232496      * | TCGA-Z7-A8R5-10A-01D..
  [2198]        8    617667-145232496      * | TCGA-Z7-A8R5-01A-42D..
  [2199]        8 125418855-145232496      * | TCGA-Z7-A8R6-10A-01D..
  [2200]        8 114820195-145232496      * | TCGA-Z7-A8R6-01A-11D..
         Num_Probes Segment_Mean
 

### b. Find the mean of the `Segment_Mean` values for copy number segments that have *any* overlap with the region chr17:37,842,337-37,886,915. (4 points)

In [26]:
myGRange17 <- GRanges(seqnames = "17",
                    ranges = IRanges(start = 37842337, end = 37886915))
segs.overlap <- find_overlaps(segs.gr, myGRange17)  %>%
    as_tibble()

head(segs.overlap)


seqnames,start,end,width,strand,Sample,Num_Probes,Segment_Mean
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>,<dbl>
17,987221,73296953,72309733,*,TCGA-3C-AAAU-10A-01D-A41E-01,33859,0.0088
17,25270517,73296953,48026437,*,TCGA-3C-AAAU-01A-11D-A41E-01,24226,0.1856
17,987221,80917016,79929796,*,TCGA-3C-AALI-10A-01D-A41E-01,36977,0.0057
17,37341549,38148341,806793,*,TCGA-3C-AALI-01A-11D-A41E-01,369,2.4574
17,987221,65395076,64407856,*,TCGA-3C-AALJ-10A-01D-A41E-01,29192,0.0067
17,35391467,40360643,4969177,*,TCGA-3C-AALJ-01A-31D-A41E-01,2305,0.6738


In [7]:
segment_mean_GRange17 <- mean(segs.overlap$Segment_Mean)
print(segment_mean_GRange17)

[1] 0.1420197


### c. Find the patient sample distribution of copy number for `PIK3CA` (hg19). (6 points)
Find the counts of samples with deletion (D; `Segment_Mean < -0.3`), neutral (N; `Segment_Mean >= -0.3 & Segment_Mean <= 0.3`), gain (G; `Segment_Mean > 0.3`) segments that have `any` overlap with `PIK3CA` gene coordinates.  


In [27]:
PIK3CA <- GRanges(seqnames = "3",
                    ranges = IRanges(start = 179148357, end = 179240093))

segs.overlap_PIK3CA <- find_overlaps(segs.gr, PIK3CA)  %>%
    as_tibble()

head(segs.overlap_PIK3CA)

seqnames,start,end,width,strand,Sample,Num_Probes,Segment_Mean
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>,<dbl>
3,89269219,197538677,108269459,*,TCGA-3C-AAAU-10A-01D-A41E-01,55921,-0.0034
3,98972306,197538677,98566372,*,TCGA-3C-AAAU-01A-11D-A41E-01,52806,0.3057
3,88436220,197538677,109102458,*,TCGA-3C-AALI-10A-01D-A41E-01,56401,-0.0030
3,93734671,197538677,103804007,*,TCGA-3C-AALI-01A-11D-A41E-01,55460,0.3440
3,161841018,181064228,19223211,*,TCGA-3C-AALJ-10A-01D-A41E-01,9398,0.0054
3,155525798,195032856,39507059,*,TCGA-3C-AALJ-01A-31D-A41E-01,20796,-0.0621


In [9]:
PIK3CA_D <- segs.overlap_PIK3CA  %>% 
    filter( Segment_Mean < -0.3) 
PIK3CA_N <- segs.overlap_PIK3CA  %>% 
    filter(Segment_Mean >= -0.3 & Segment_Mean <= 0.3) 
PIK3CA_G <- segs.overlap_PIK3CA  %>% 
    filter(Segment_Mean > -0.3) 

print(paste("Number of samples with deletion: ", nrow(PIK3CA_D)))
print(paste("Number of samples with neutral: ", nrow(PIK3CA_N)))
print(paste("Number of samples with gain: ", nrow(PIK3CA_G)))

[1] "Number of samples with deletion:  14"
[1] "Number of samples with neutral:  2024"
[1] "Number of samples with gain:  2189"


# Problem 2: Frequency of copy number alteration events within genomic regions. (12 points total) 

This problem will continue to use the copy number data stored in `segs.gr`.

### a. Create a genome-wide tile of 1Mb windows for the human genome (`hg19`). (6 points)
See *3.1 Tiling the genome* of *Lecture16_GenomicData.Rmd* for hints.


In [10]:
seqinfo <- Seqinfo(genome = "hg19")
seqinfo <- keepStandardChromosomes(seqinfo) 
seqlevelsStyle(seqinfo) <- "NCBI"
seqinfo

Warning message in (function (seqlevels, genome, new_style) :
"cannot switch some hg19's seqlevels from UCSC to NCBI style"


Seqinfo object with 25 sequences (1 circular) from 2 genomes (GRCh37.p13, hg19):
  seqnames seqlengths isCircular     genome
  1         249250621      FALSE GRCh37.p13
  2         243199373      FALSE GRCh37.p13
  3         198022430      FALSE GRCh37.p13
  4         191154276      FALSE GRCh37.p13
  5         180915260      FALSE GRCh37.p13
  ...             ...        ...        ...
  21         48129895      FALSE GRCh37.p13
  22         51304566      FALSE GRCh37.p13
  X         155270560      FALSE GRCh37.p13
  Y          59373566      FALSE GRCh37.p13
  chrM          16571       TRUE       hg19

In [11]:
slen <- seqlengths(seqinfo) 
tileWidth <- 1000000 
tiles <- tileGenome(seqlengths = slen, tilewidth = tileWidth,
                    cut.last.tile.in.chrom = TRUE)
tiles

GRanges object with 3114 ranges and 0 metadata columns:
         seqnames            ranges strand
            <Rle>         <IRanges>  <Rle>
     [1]        1         1-1000000      *
     [2]        1   1000001-2000000      *
     [3]        1   2000001-3000000      *
     [4]        1   3000001-4000000      *
     [5]        1   4000001-5000000      *
     ...      ...               ...    ...
  [3110]        Y 56000001-57000000      *
  [3111]        Y 57000001-58000000      *
  [3112]        Y 58000001-59000000      *
  [3113]        Y 59000001-59373566      *
  [3114]     chrM           1-16571      *
  -------
  seqinfo: 25 sequences from an unspecified genome

### b. Find the 1Mb window with the most frequent overlapping deletions. (6 points)
Find the 1Mb windows with `any` overlap with deletion copy number segments. Assume a deletion segment is defined as a segment in `segs.gr` having `Segment_Mean < -0.3`. 

Return one of the 1Mb window `Granges` entry with the highest frequency (count) of deletion segments.

Hint: Subset the `segs.gr` to only rows with `Segment_Mean < -0.3`. 

In [28]:
segs.overlap_1mb <- find_overlaps(segs.gr, tiles)  %>%
    as_tibble()
head(segs.overlap_1mb)

seqnames,start,end,width,strand,Sample,Num_Probes,Segment_Mean
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>,<dbl>
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055
1,3218610,95674710,92456101,*,TCGA-3C-AAAU-10A-01D-A41E-01,53225,0.0055


In [29]:
deletions_1mb <- segs.overlap_1mb  %>% 
    filter( Segment_Mean < -0.3)  %>% 
    arrange(Segment_Mean) 

head(deletions_1mb)

seqnames,start,end,width,strand,Sample,Num_Probes,Segment_Mean
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>,<dbl>
17,5960849,5961137,289,*,TCGA-BH-A5J0-01A-11D-A27O-01,2,-7.5883
8,11309849,11309893,45,*,TCGA-A2-A1FW-01A-11D-A13J-01,2,-7.5751
17,11835078,11839540,4463,*,TCGA-A8-A08S-01A-11D-A036-01,5,-7.5132
16,66036144,66036359,216,*,TCGA-A7-A0CJ-01A-21D-A011-01,2,-7.5064
17,17411211,17415217,4007,*,TCGA-BH-A0DD-01A-31D-A12N-01,3,-7.4793
1,20561069,20561095,27,*,TCGA-AC-A3EH-01A-22D-A227-01,2,-7.4771


In [30]:
deletions_1mb[1,]

seqnames,start,end,width,strand,Sample,Num_Probes,Segment_Mean
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>,<dbl>
17,5960849,5961137,289,*,TCGA-BH-A5J0-01A-11D-A27O-01,2,-7.5883


# Problem 3: Reading and annotating genomic variants (16 points total)

### Preparation

In [15]:
vcfFile <- "/Users/sgrizzi/Desktop/TFCB_data/GIAB_highconf_v.3.3.2.vcf.gz"
vcfHead <- scanVcfHeader(vcfFile)
myGRange8 <- GRanges(seqnames = "8", ranges = IRanges(start = 128700000, end = 129000000))
vcf.param <- ScanVcfParam(which = myGRange8) 

### a. Load variant data from VCF file `GIAB_highconf_v.3.3.2.vcf.gz` for `chr8:128,700,000-129,000,000`. (4 points)
Note: use genome build `hg19`.

In [31]:
vcf <- readVcf(vcfFile, genome = "hg19", param = vcf.param)

### b. Combine the fields of the VCF genotype information into a table. (4 points)
You may use your choice of data objects (e.g. `data.frame`).

In [ ]:
rowRanges(vcf)

In [18]:
geno(header(vcf)) %>%
  as.data.frame() %>%
  rownames_to_column("colname")

colname,Number,Type,Description
<chr>,<chr>,<chr>,<chr>
GT,1,String,Consensus Genotype across all datasets with called genotype
DP,1,Integer,"Total read depth summed across all datasets, excluding MQ0 reads"
GQ,1,Integer,"Net Genotype quality across all datasets, calculated from GQ scores of callsets supporting the consensus GT, using only one callset from each dataset"
ADALL,R,Integer,Net allele depths across all datasets
AD,R,Integer,Net allele depths across all unfiltered datasets with called genotype
IGT,1,String,Original input genotype
IPS,1,String,Phase set for IGT
PS,1,String,Phase set for GT


### c. Retrieve the following information at chr8:128747953. (8 points)
Print out the SNP ID (i.e. "rs ID"), reference base (`REF`), alterate base (`ALT`), genotype (`GT`), depth (`DP`), allele depth (`ADALL`), phase set (`PS`).

Hints: 

  i. `REF` and `ALT` are in the output of `rowRanges(vcf)`. See Section `3a` in `Lecture16_VariantCalls.ipynb` 
  ii. To get the sequence of `DNAString`, use `as.character(x)`.  
  ii. To get the sequence of `DNAStringSet`, use `as.character(unlist(x))`. 
  iii. To expand a list of information for `geno`, use `unlist(x)`.  

  

In [19]:
myGRange8 <- GRanges(seqnames = "8",
                    ranges = IRanges(start = 128747953, end = 128747953))
vcf.param8 <- ScanVcfParam(which = myGRange8) 
vcf8 <- readVcf(vcfFile, genome = "hg19", param = vcf.param8) 

In [20]:
vcf8_seq <- rowRanges(vcf8)  %>% 
    as.data.frame()  %>% 
    dplyr::select(c(REF, ALT))
vcf8_seq

,REF,ALT
,<chr>,<list>
rs3824120,G,T


In [21]:
genoData8 <- bind_cols(lapply(geno(vcf8), as.data.frame))
colnames(genoData8) <- rownames(geno(header(vcf8)))
genoData8

New names:
* `HG001` -> `HG001...1`
* `HG001` -> `HG001...2`
* `HG001` -> `HG001...3`
* `HG001` -> `HG001...4`
* `HG001` -> `HG001...5`
* `HG001` -> `HG001...6`
* `HG001` -> `HG001...7`
* `HG001` -> `HG001...8`


,GT,DP,GQ,ADALL,AD,IGT,IPS,PS
,<chr>,<int>,<int>,<list>,<list>,<chr>,<chr>,<chr>
rs3824120,0|1,461,668,"105, 94","128, 121",0/1,.,PATMAT


In [22]:
SNP_8 <- cbind(genoData8, vcf8_seq)
SNP_8

,GT,DP,GQ,ADALL,AD,IGT,IPS,PS,REF,ALT
,<chr>,<int>,<int>,<list>,<list>,<chr>,<chr>,<chr>,<chr>,<list>
rs3824120,0|1,461,668,"105, 94","128, 121",0/1,.,PATMAT,G,T
